In [5]:
import pandas as pd
import ast

In [3]:
hit_id = open('hit_id.txt','r').read()
#x = pd.read_json(hit_id + '_demo.txt')

In [26]:
data = ast.literal_eval(open('../subject_data/pilot_data/3N1FSUEFL6KRV7KU23SCJYJFFQA4DF_demo.txt','r').read())
data2 = ast.literal_eval(open('../subject_data/pilot_data/3CTOC39K38A1HG25EXCUP8Y2BCXJ7S_demo.txt','r').read())
print data
print data2

{'screen_width': 1440, 'window_height': 675, 'workerId': 'A3PC2BR0YGEBA', 'age': '21', 'totalTime_mins': 24.5, 'comments': '', 'window_width': 1390, 'curId': '3N1FSUEFL6KRV7KU23SCJYJFFQA4DF', 'race': 'white', 'vision': 'normal', 'curTime': '24/09/2018@12:48:35', 'gender': 'female', 'ethnicity': 'not_hispanic', 'screen_height': 900}
{'screen_width': 1920, 'window_height': 675, 'workerId': 'A1NEJS4NPQUA89', 'age': '20', 'totalTime_mins': 26.22355, 'comments': '- Eleanor (Ellie) Pinchbeck\n- I didn\'t encounter any bugs but I noticed that in the last phase of the experiment, sometimes the categories of "color" and "shape" appeared before or at the same time as the stimuli appeared\n- I believe the instructions were clear\n- I took me a little under 30 minutes (~28)\n- N/A', 'window_width': 1855, 'curId': '3CTOC39K38A1HG25EXCUP8Y2BCXJ7S', 'race': 'white', 'vision': 'impaired', 'curTime': '24/09/2018@13:41:37', 'gender': 'female', 'ethnicity': 'not_hispanic', 'screen_height': 1080}


In [40]:
data = ast.literal_eval(open('../subject_data/pilot_data/3N1FSUEFL6KRV7KU23SCJYJFFQA4DF_cued.txt','r').read())
data2 = ast.literal_eval(open('../subject_data/pilot_data/3CTOC39K38A1HG25EXCUP8Y2BCXJ7S_cued.txt','r').read())

In [31]:
def combineDicts(x,y):
    out = {}
    for e in x:
        out[e] = [x[e], y[e]]
    return out

aggData = combineDicts(data,data2)

In [35]:
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

print take(5, aggData.iteritems())

[('finishTime', [330026, 317851]), ('screen_width', [1440, 1920]), ('window_height', [708, 708]), ('workerId', ['A3PC2BR0YGEBA', 'A1NEJS4NPQUA89']), ('window_width', [1390, 1855])]


In [36]:
pd.DataFrame.from_dict(aggData)

,counterBalance_keys,counterBalance_order,curId,curTime,finishTime,screen_height,screen_width,trialStruct,userAgent,window_height,window_width,workerId
0,"{""68"":""red_key"",""70"":""blue_key"",""74"":""circle_k...",cued_first,3N1FSUEFL6KRV7KU23SCJYJFFQA4DF,24/09/2018@12:34:13,330026,900,1440,"[{u'rt': 1060, u'stim_color': u'red', u'transi...",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_3...,708,1390,A3PC2BR0YGEBA
1,"{""68"":""blue_key"",""70"":""red_key"",""74"":""circle_k...",rvts_first,3CTOC39K38A1HG25EXCUP8Y2BCXJ7S,24/09/2018@13:38:40,317851,1080,1920,"[{u'rt': 1375, u'stim_color': u'blue', u'trans...",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6...,708,1855,A1NEJS4NPQUA89


## Nope
The method above clearly can't handle nested dictionaries

In [120]:
data = ast.literal_eval(open('../subject_data/pilot_data/3N1FSUEFL6KRV7KU23SCJYJFFQA4DF_demo.txt','r').read())
data2 = ast.literal_eval(open('../subject_data/pilot_data/3CTOC39K38A1HG25EXCUP8Y2BCXJ7S_cued.txt','r').read())

In [132]:
if 'counterBalance_keys' in data.keys(): data['counterBalance_keys'] = ast.literal_eval(data['counterBalance_keys']) 
print data.keys()

['screen_width', 'window_height', 'workerId', 'age', 'totalTime_mins', 'comments', 'window_width', 'curId', 'race', 'vision', 'curTime', 'gender', 'ethnicity', 'screen_height']


In [134]:
## make headers
headers = data.keys()
if 'counterBalance_keys' in data.keys(): headers += data['counterBalance_keys'].keys()
    
if 'blockStruct' in data.keys():
     headers += data['blockStruct'][0].keys() + data['blockStruct'][0]['trialStruct'][0].keys()
elif 'trialStruct' in data.keys():
    headers += data['trialStruct'][0].keys()

headers = [x for x in headers if x not in ['trialStruct', 'counterBalance_keys']]

print headers

['screen_width', 'window_height', 'workerId', 'age', 'totalTime_mins', 'comments', 'window_width', 'curId', 'race', 'vision', 'curTime', 'gender', 'ethnicity', 'screen_height']


In [136]:
def summarizeSubject(e):
    
    subjectLevel = [e[x] for x in e if x not in ['trialStruct', 'blockStruct', 'counterBalance_keys']]
    if 'counterBalance_keys' in e.keys(): subjectLevel += e['counterBalance_keys'].values()
    trials = []
    build = []
    
    if 'blockStruct' in e.keys():
        for block in e['blockStruct']:
            for trial in block['trialStruct']:
                trials.append(subjectLevel + [block[x] for x in block if x not in 'trialStruct'] + [x for x in trial.values()])
                
    elif 'trialStruct' in e.keys():
        for trial in e['trialStruct']:
            trials.append(subjectLevel + trial.values())
            
    else:
        trials.append(subjectLevel)
    
    return trials

print summarizeSubject(data)[:10]

[[1440, 675, 'A3PC2BR0YGEBA', '21', 24.5, '', 1390, '3N1FSUEFL6KRV7KU23SCJYJFFQA4DF', 'white', 'normal', '24/09/2018@12:48:35', 'female', 'not_hispanic', 900]]
